# Pacing data

Experiments vs simulations.

In [ ]:
using ModelingToolkit
using OrdinaryDiffEq, SteadyStateDiffEq, DiffEqCallbacks
using Plots
using CSV
using DataFrames
using LsqFit
using CaMKIIModel
using CaMKIIModel: second
Plots.default(lw=1.5)

## Pacing duration and CaMKII activity

### Experiments

30 seconds resting + N seconds 1Hz pacing + resting.

In [ ]:
durationdf = CSV.read(joinpath(@__DIR__, "data/CaMKAR-duration.csv"), DataFrame)
ts = durationdf[!, "Time(sec)"]
fifteen = durationdf[!, "1Hz 15sec (Mean)"]
fifteen_error = durationdf[!, "1Hz 15sec (SD)"] ./ sqrt.(durationdf[!, "1Hz 15sec (N)"])
thirty = durationdf[!, "1Hz 30sec (Mean)"] .+ 0.25
thirty_error = durationdf[!, "1Hz 30sec (SD)"] ./ sqrt.(durationdf[!, "1Hz 30sec (N)"])
sixty = durationdf[!, "1Hz 60sec (Mean)"]
sixty_error = durationdf[!, "1Hz 60sec (SD)"] ./ sqrt.(durationdf[!, "1Hz 60sec (N)"])
ninety = durationdf[!, "1Hz 90sec (Mean)"] .- 0.25
ninety_error = durationdf[!, "1Hz 90sec (SD)"] ./ sqrt.(durationdf[!, "1Hz 90sec (N)"])

## 30 sec timeseries +0.25 and 90 sec timeseries -0.25 for a consistent baseline before pacing.
plot(ts, fifteen, yerr=fifteen_error, lab="15 sec", color=:blue, markerstrokecolor=:blue)
plot!(ts, thirty, yerr=thirty_error, lab="30 sec", color=:red, markerstrokecolor=:red)
plot!(ts, sixty, yerr=sixty_error, lab="60 sec", color=:orange, markerstrokecolor=:orange)
plot!(ts, ninety, yerr=ninety_error, lab="90 sec", color=:green, markerstrokecolor=:green)
plot!(title="Experiment", xlabel="Time (s)", ylabel="CaMKII activity (AU)")

In [ ]:
savefig("pacing-duration-exp.pdf")

### Simulation

In [ ]:
@time "Build system" sys = build_neonatal_ecc_sys(simplify=true, reduce_iso=true, reduce_camk=true)
tend = 500second
@time "Build problem" prob = ODEProblem(sys, [sys.kdeph_CaMK => inv(10second)], tend)
@time "Build problem" prob_n0a2 = ODEProblem(sys, [sys.k_P1_P2=>0, sys.kdeph_CaMK => inv(12second)], tend)
stimstart = 100second
stimend = 300second
@unpack Istim = sys
alg = KenCarp47()

In [ ]:
stimstart = 30second
callback15 = build_stim_callbacks(Istim, stimstart + 15second; period=1second, starttime=stimstart)
sol15 = solve(prob, alg; callback=callback15)
sol15_n0a2 = solve(prob_n0a2, alg; callback=callback15)
callback30 = build_stim_callbacks(Istim, stimstart + 30second; period=1second, starttime=stimstart)
sol30 = solve(prob, alg; callback=callback30)
sol30_n0a2 = solve(prob_n0a2, alg; callback=callback30)
callback60 = build_stim_callbacks(Istim, stimstart + 60second; period=1second, starttime=stimstart)
sol60 = solve(prob, alg; callback=callback60)
sol60_n0a2 = solve(prob_n0a2, alg; callback=callback60)
callback90 = build_stim_callbacks(Istim, stimstart + 90second; period=1second, starttime=stimstart)
sol90 = solve(prob, alg; callback=callback90)
sol90_n0a2 = solve(prob_n0a2, alg; callback=callback90)
idxs = (sys.t / 1000, sys.CaMKAct * 100)

plot(sol15, idxs=idxs, tspan=(0second, 205second), lab="15 sec", color=:blue)
plot!(sol30, idxs=idxs, tspan=(0second, 205second), lab="30 sec", color=:red)
plot!(sol60, idxs=idxs, tspan=(0second, 205second), lab="60 sec", color=:orange)
plot!(sol90, idxs=idxs, tspan=(0second, 205second), lab="90 sec", color=:green)
plot!(title="Simulation", xlabel="Time (s)", ylabel="CaMKII activity (%)")

In [ ]:
savefig("pacing-duration-sim.pdf")

### Decay rates

Fit against an exponential decay model.

In [ ]:
@. decay_model(x, p) = p[1] * exp(-x / p[2]) + p[3]
rmse(fit) = sqrt(sum(abs2, fit.resid) / length(fit.resid))

In [ ]:
xdata_15 = ts[10:20] .- ts[10]
ydata_15 = fifteen[10:20]
xdata_30 = ts[13:23] .- ts[13]
ydata_30 = thirty[13:23]
xdata_60 = ts[19:29] .- ts[19]
ydata_60 = sixty[19:29]
xdata_90 = ts[25:35] .- ts[25]
ydata_90 = ninety[25:35]

In [ ]:
ysim_15 = sol15(stimstart+15second:5second:stimstart+15second+50second ; idxs=sys.CaMKAct * 100).u
ysim_30 = sol30(stimstart+30second:5second:stimstart+30second+50second ; idxs=sys.CaMKAct * 100).u
ysim_60 = sol60(stimstart+60second:5second:stimstart+60second+50second ; idxs=sys.CaMKAct * 100).u
ysim_90 = sol90(stimstart+90second:5second:stimstart+90second+50second ; idxs=sys.CaMKAct * 100).u

ysim_15_noa2 = sol15_n0a2(stimstart+15second:5second:stimstart+15second+50second ; idxs=sys.CaMKAct * 100).u
ysim_30_noa2 = sol30_n0a2(stimstart+30second:5second:stimstart+30second+50second ; idxs=sys.CaMKAct * 100).u
ysim_60_noa2 = sol60_n0a2(stimstart+60second:5second:stimstart+60second+50second ; idxs=sys.CaMKAct * 100).u
ysim_90_noa2 = sol90_n0a2(stimstart+90second:5second:stimstart+90second+50second ; idxs=sys.CaMKAct * 100).u

In [ ]:
p0 = [1.0, 2.0, 13.0]

# Experimental fits
fit_15 = curve_fit(decay_model, xdata_15, ydata_15, p0, autodiff=:forwarddiff)
fit_30 = curve_fit(decay_model, xdata_30, ydata_30, p0, autodiff=:forwarddiff)
fit_60 = curve_fit(decay_model, xdata_60, ydata_60, p0, autodiff=:forwarddiff)
fit_90 = curve_fit(decay_model, xdata_90, ydata_90, p0, autodiff=:forwarddiff)

# Simulation fits
fit_sim_15 = curve_fit(decay_model, xdata_15, ysim_15, p0, autodiff=:forwarddiff)
fit_sim_30 = curve_fit(decay_model, xdata_30, ysim_30, p0, autodiff=:forwarddiff)
fit_sim_60 = curve_fit(decay_model, xdata_60, ysim_60, p0, autodiff=:forwarddiff)
fit_sim_90 = curve_fit(decay_model, xdata_90, ysim_90, p0, autodiff=:forwarddiff)

# Simulation fits (CaMKII A2 removed)
fit_sim_15_noa2 = curve_fit(decay_model, xdata_15, ysim_15_noa2, p0, autodiff=:forwarddiff)
fit_sim_30_noa2 = curve_fit(decay_model, xdata_30, ysim_30_noa2, p0, autodiff=:forwarddiff)
fit_sim_60_noa2 = curve_fit(decay_model, xdata_60, ysim_60_noa2, p0, autodiff=:forwarddiff)
fit_sim_90_noa2 = curve_fit(decay_model, xdata_90, ysim_90_noa2, p0, autodiff=:forwarddiff)

In [ ]:
println("The time scale for experimental: ")
for (fit, dur) in zip((fit_15, fit_30, fit_60, fit_90), (15, 30, 60, 90))
    println("  $dur sec pacing is $(round(coef(fit)[2]; digits=2)) seconds.")
end

println("The time scale for simulation: ")
for (fit, dur) in zip((fit_sim_15, fit_sim_30, fit_sim_60, fit_sim_90), (15, 30, 60, 90))
    println("  $dur sec pacing is $(round(coef(fit)[2]; digits=2)) seconds.")
end

println("The time scale for simulation without CaMKII A2: ")
for (fit, dur) in zip((fit_sim_15_noa2, fit_sim_30_noa2, fit_sim_60_noa2, fit_sim_90_noa2), (15, 30, 60, 90))
    println("  $dur sec pacing is $(round(coef(fit)[2]; digits=2)) seconds.")
end

In [ ]:
p1 = plot(xdata_15, ydata_15, label="Exp 15 sec")
plot!(xdata_15, decay_model(xdata_15, coef(fit_15)), label="Fit Exp 15 sec", linestyle=:dash)
p2 = plot(xdata_30, ydata_30, label="Exp 30 sec")
plot!(xdata_30, decay_model(xdata_30, coef(fit_30)), label="Fit Exp 30 sec", linestyle=:dash)
p3 = plot(xdata_60, ydata_60, label="Exp 60 sec")
plot!(xdata_60, decay_model(xdata_60, coef(fit_60)), label="Fit Exp 60 sec", linestyle=:dash)
p4 = plot(xdata_90, ydata_90, label="Exp 90 sec")
plot!(xdata_90, decay_model(xdata_90, coef(fit_90)), label="Fit Exp 90 sec", linestyle=:dash)

plot(p1, p2, p3, p4; layout=(2,2), xlabel="Time (s)", ylabel="CaMKII activity (AU)")

In [ ]:
savefig("pacing-decay-fitting.pdf")

In [ ]:
p1 = plot(xdata_15, ysim_15, label="Sim 15 sec")
plot!(xdata_15, decay_model(xdata_15, coef(fit_sim_15)), label="Fit Sim 15 sec", linestyle=:dash)
p2 = plot(xdata_30, ysim_30, label="Sim 30 sec")
plot!(xdata_30, decay_model(xdata_30, coef(fit_sim_30)), label="Fit Sim 30 sec", linestyle=:dash)
p3 = plot(xdata_60, ysim_60, label="Sim 60 sec")
plot!(xdata_60, decay_model(xdata_60, coef(fit_sim_60)), label="Fit Sim 60 sec", linestyle=:dash)
p4 = plot(xdata_90, ysim_90, label="Sim 90 sec")
plot!(xdata_90, decay_model(xdata_90, coef(fit_sim_90)), label="Fit Sim 90 sec", linestyle=:dash)

plot(p1, p2, p3, p4; layout=(2,2), xlabel="Time (s)", ylabel="CaMKII activity (AU)")

In [ ]:
p1 = plot(xdata_15, ysim_15_noa2, label="Sim 15 sec")
plot!(xdata_15, decay_model(xdata_15, coef(fit_sim_15_noa2)), label="Fit Sim 15 sec", linestyle=:dash)
p2 = plot(xdata_30, ysim_30_noa2, label="Sim 30 sec")
plot!(xdata_30, decay_model(xdata_30, coef(fit_sim_30_noa2)), label="Fit Sim 30 sec", linestyle=:dash)
p3 = plot(xdata_60, ysim_60_noa2, label="Sim 60 sec")
plot!(xdata_60, decay_model(xdata_60, coef(fit_sim_60_noa2)), label="Fit Sim 60 sec", linestyle=:dash)
p4 = plot(xdata_90, ysim_90_noa2, label="Sim 90 sec")
plot!(xdata_90, decay_model(xdata_90, coef(fit_sim_90_noa2)), label="Fit Sim 90 sec", linestyle=:dash)

plot(p1, p2, p3, p4; layout=(2,2), xlabel="Time (s)", ylabel="CaMKII activity (AU)")

In [ ]:
plot([15, 30, 60, 90], [coef(fit_15)[2], coef(fit_30)[2], coef(fit_60)[2], coef(fit_90)[2]], marker=:circle, label="Exp")
plot!([15, 30, 60, 90], [coef(fit_sim_15)[2], coef(fit_sim_30)[2], coef(fit_sim_60)[2], coef(fit_sim_90)[2]], marker=:diamond, label="Sim")
plot!([15, 30, 60, 90], [coef(fit_sim_15_noa2)[2], coef(fit_sim_30_noa2)[2], coef(fit_sim_60_noa2)[2], coef(fit_sim_90_noa2)[2]], marker=:star5, label="Sim without A2")
plot!(title="Decay Time Scale vs Pacing Duration", xlabel="Pacing Duration (s)", ylabel="Decay Time Scale (s)", xlims=(0, 100), legend=:top)

In [ ]:
savefig("pacing-decay-exp-sim.pdf")

### Phosphorylated fraction

In [ ]:
idxs = (sys.t / 1000, (sys.CaMKP + sys.CaMKA + sys.CaMKA2) * 100)
plot(sol15, idxs=idxs, tspan=(0second, 205second), lab="15 sec", color=:blue)
plot!(sol30, idxs=idxs, tspan=(0second, 205second), lab="30 sec", color=:red)
plot!(sol60, idxs=idxs, tspan=(0second, 205second), lab="60 sec", color=:orange)
plot!(sol90, idxs=idxs, tspan=(0second, 205second), lab="90 sec", color=:green)
plot!(title="Simulation", xlabel="Time (s)", ylabel="Phosphorylated CaMKII (%)")

In [ ]:
savefig("pacing-duration-phos.pdf")

## Pacing frequency and CaMKII activity

### Experiments

In [ ]:
freqdf = CSV.read(joinpath(@__DIR__, "data/CaMKAR-freq.csv"), DataFrame)
ts = 0:5:205
onehz = freqdf[!, "1Hz (Mean)"]
onehz_error = freqdf[!, "1Hz (SD)"] ./ sqrt.(freqdf[!, "1Hz (N)"])
twohz = freqdf[!, "2Hz (Mean)"]
twohz_error = freqdf[!, "2Hz (SD)"] ./ sqrt.(freqdf[!, "2Hz (N)"])

plot(ts, onehz, yerr=onehz_error, lab="1 Hz", color=:blue, markerstrokecolor=:blue)
plot!(ts, twohz, yerr=twohz_error, lab="2 Hz", color=:red, markerstrokecolor=:red)
plot!(title="Experiment", xlabel="Time (s)", ylabel="CaMKII activity (AU)")

In [ ]:
savefig("pacing-frequency-exp.pdf")

### Simulations

In [ ]:
tend = 205.0second
prob = ODEProblem(sys, [], tend)
stimstart = 30.0second
stimend = 120.0second
callback = build_stim_callbacks(Istim, stimend; period=1second, starttime=stimstart)
sol1 = solve(prob, alg; callback)

callback2 = build_stim_callbacks(Istim, stimend; period=0.5second, starttime=stimstart)
sol2 = solve(prob, alg; callback=callback2)
idxs = (sys.t / 1000, sys.CaMKAct * 100)

plot(sol1, idxs=idxs, lab="1 Hz", color=:blue)
plot!(sol2, idxs=idxs, lab="2 Hz", color=:red)
plot!(title="Simulation", xlabel="Time (s)", ylabel="CaMKII activity (%)")

In [ ]:
savefig("pacing-frequency-sim.pdf")

In [ ]:
idxs = (sys.t / 1000, (sys.CaMKP + sys.CaMKA + sys.CaMKA2) * 100)
plot(sol1, idxs=idxs, lab="1 Hz", color=:blue)
plot!(sol2, idxs=idxs, lab="2 Hz", color=:red)
plot!(title="Simulation", xlabel="Time (s)", ylabel="Phosphorylated CaMKII (%)")

In [ ]:
savefig("pacing-frequency-phos.pdf")